<a href="https://colab.research.google.com/github/Arshdeep-Sandhu210/MovieRecomendation/blob/main/blas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
# Step 2: Create C code file
%%writefile blas_sgemm.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cblas.h>

#define M 1000  // Rows of A and C
#define N 1000  // Columns of B and C
#define K 2000  // Columns of A and rows of B

void fill_random(float *matrix, int rows, int cols) {
    for (int i = 0; i < rows * cols; ++i) {
        matrix[i] = (float)rand() / RAND_MAX * 10.0f;
    }
}

int main() {
    srand(time(NULL));

    float alpha = 1.0f, beta = 0.0f;

    float* A = (float*)malloc(M * K * sizeof(float));
    float* B = (float*)malloc(K * N * sizeof(float));
    float* C = (float*)malloc(M * N * sizeof(float));

    if (!A || !B || !C) {
        printf("Memory allocation failed!\n");
        return 1;
    }

    fill_random(A, M, K);
    fill_random(B, K, N);

    for (int i = 0; i < M * N; ++i) {
        C[i] = 0.0f;
    }

    // Time just the multiplication
    clock_t start = clock();

    cblas_sgemm(
        CblasRowMajor, CblasNoTrans, CblasNoTrans,
        M, N, K,
        alpha, A, K,
        B, N,
        beta, C, N
    );

    clock_t end = clock();
    double elapsed_secs = (double)(end - start) / CLOCKS_PER_SEC;

    printf("Multiplication took %.6f seconds\n", elapsed_secs);

    free(A);
    free(B);
    free(C);

    return 0;
}



Overwriting blas_sgemm.c


In [55]:
!gcc -o blas_sgemm blas_sgemm.c -lopenblas
!./blas_sgemm


Multiplication took 0.109434 seconds


In [56]:
%%writefile manual_sgemm.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define M 1000  // Rows of A and C
#define N 1000  // Columns of B and C
#define K 2000  // Columns of A and rows of B

void fill_random(float *matrix, int rows, int cols) {
    for (int i = 0; i < rows * cols; ++i) {
        matrix[i] = (float)rand() / RAND_MAX * 10.0f;
    }
}

int main() {
    srand(time(NULL));

    float* A = (float*)malloc(M * K * sizeof(float));
    float* B = (float*)malloc(K * N * sizeof(float));
    float* C = (float*)malloc(M * N * sizeof(float));

    if (!A || !B || !C) {
        printf("Memory allocation failed!\n");
        return 1;
    }

    fill_random(A, M, K);
    fill_random(B, K, N);

    for (int i = 0; i < M * N; ++i) {
        C[i] = 0.0f;
    }

    // Time just the multiplication
    clock_t start = clock();

    // Manual matrix multiplication: C = A * B
    for (int i = 0; i < M; ++i) {
        for (int j = 0; j < N; ++j) {
            float sum = 0.0f;
            for (int p = 0; p < K; ++p) {
                sum += A[i * K + p] * B[p * N + j];
            }
            C[i * N + j] = sum;
        }
    }

    clock_t end = clock();
    double elapsed_secs = (double)(end - start) / CLOCKS_PER_SEC;

    printf("Manual multiplication took %.6f seconds\n", elapsed_secs);

    free(A);
    free(B);
    free(C);

    return 0;
}


Overwriting manual_sgemm.c


In [57]:
!gcc -O2 -o manual_sgemm manual_sgemm.c
!./manual_sgemm


Manual multiplication took 21.549966 seconds
